In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.insert(0, "../src")
from pathlib import Path

import numpy as np
from joblib import Parallel, delayed

import constants
from retrieval.fever_doc_db import FeverDocDB
from gen import util

In [4]:
fever_dev = util.read_data(Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/bert-data/fever.dev.n5.nei.jsonl"))
len(fever_dev)

9999

In [2]:
a = [
    [1,2], [3,4]
]
b = [
    [2,1],[1,2]
]

a[0] in b[1]

False

# Playground

In [46]:
import torch
from datasets import Dataset, load_metric, DatasetDict
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig

checkpoint = "bert-base-uncased"
conf = AutoConfig.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, id2label=constants.ID2LABEL, label2id=constants.LABEL2ID)
model.train()

loading configuration file config.json from cache at /users/k21190024/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2eb64c8a40076/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file config.json from cache at /users/k21190024/.cache/huggingface/hub/models--bert-base-uncased/snapshots/1dbc166cf8765166998eff31ade2

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [48]:
def preprocess(examples):
    return tokenizer(examples["claim"], examples["evidence"], truncation=True, padding=True, max_length=512, return_tensors="pt")

data = DatasetDict({
    "train": Dataset.from_list(fever_dev[:1000]), 
    "dev": Dataset.from_list(fever_dev[1000:1200])
}).map(preprocess, batched=True).rename_column("label", "labels")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [49]:
metric = load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [50]:
metric_name = "accuracy"
model_name = "./tmp"
batch_size = 16

args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_strategy='epoch',
    metric_for_best_model="accuracy",
    push_to_hub=False,
    seed=constants.SEED
)

trainer = Trainer(
    model,
    args,
    train_dataset=data["train"],
    eval_dataset=data["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneo

Epoch,Training Loss,Validation Loss,Accuracy
1,0.840700,0.612331,0.645000
2,0.493700,0.546812,0.700000
3,0.370400,0.553392,0.730000
4,0.235800,0.537465,0.755000
5,0.155800,0.542134,0.785000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
Saving model checkpoint to ./tmp/checkpoint-63
Configuration saved in ./tmp/checkpoint-63/config.json
Model weights saved in ./tmp/checkpoint-63/pytorch_model.bin
tokenizer config file saved in ./tmp/checkpoint-63/tokenizer_config.json
Special tokens file saved in ./tmp/checkpoint-63/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: evidence, claim. If evidence, claim are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch s

TrainOutput(global_step=315, training_loss=0.4192861617557586, metrics={'train_runtime': 127.6384, 'train_samples_per_second': 39.173, 'train_steps_per_second': 2.468, 'total_flos': 1315567088640000.0, 'train_loss': 0.4192861617557586, 'epoch': 5.0})

In [51]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [66]:
model(**preprocess(fever_dev[1001]))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)